In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(101)

In [4]:
transform = transforms.ToTensor()

Import the MNIST Dataset

In [5]:
train_data = datasets.MNIST(root = "train_images/MNIST", train=True, download=True, transform=transform)

In [6]:
test_data = datasets.MNIST(root = "train_images/MNIST", train=False, download=True, transform=transform)

In [7]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: train_images/MNIST
    Split: Test
    StandardTransform
Transform: ToTensor()

In [8]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: train_images/MNIST
    Split: Train
    StandardTransform
Transform: ToTensor()

In [9]:
train_loader = DataLoader(dataset=train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=10, shuffle=False)

*conv1 = nn.Conv2d(1,6,3,1)*:

- 1, for one input channel (grayscale)
- 6, output channels for feature extractions, the feature maps, the filters that the CNN will figure out
- 3, kernel size for a 3 by 3 filter
- 1, for the horizontal and vertical stride of the convolution


In [10]:
conv1 = nn.Conv2d(1,6,3,1)
conv2 = nn.Conv2d(6,16,3,1)

In [11]:
for i, (X_train, Y_train) in enumerate(train_data):
    break

In [12]:
X_train.shape

torch.Size([1, 28, 28])

In [13]:
x = X_train.view(1,1,28,28)

In [14]:
out = conv1(x)

In [15]:
out.shape

torch.Size([1, 6, 26, 26])

In [16]:
max_p = F.max_pool2d(out,2,2)

In [17]:
max_p.shape

torch.Size([1, 6, 13, 13])

How to feed this output of max_p to a fully conected input?

In [22]:
x = max_p.view(-1, 6*13*13)

In [23]:
x.shape

torch.Size([1, 1014])

In [ ]:
class ConvolutionalNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,6,3,1)
        self.conv2 = nn.Conv2d(6,16,3,1)
        self.fc1 = nn.Linear(5*5*16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, X):
        X = F.relu(self.conv1(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, 2, 2)
        X = X.view(-1, 16*5*5)
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = self.fc3(X)
        return F.log_softmax(X, dim=1)